In [ ]:
# Uncomment the below command if you don't have openpyxl
#!pip install openpyxl 

In [1]:
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
market_data = pd.read_excel("../../raw_data/market_data.xlsx")

In [3]:
# Create a period for a whole day which are 48 as
# Spot prices are taken by the 30 minutes mark.
period = [48]
count = 1
for i in range(1, len(market_data) + 1):
    period.append(count)
    count += 1
    if (i % 48) == 0:
        count = 1
        
market_data['period'] = pd.Series(period)

market_data['Time (UTC+10)'] = pd.to_datetime(market_data['Time (UTC+10)'])
market_data['date'] = market_data['Time (UTC+10)'].dt.date
market_data.columns

Index(['Time (UTC+10)', 'Regions NSW Trading Price ($/MWh)',
       'Regions SA Trading Price ($/MWh)', 'Regions TAS Trading Price ($/MWh)',
       'Regions VIC Trading Price ($/MWh)',
       'Regions NSW Trading Total Intermittent Generation (MW)',
       'Regions SA Trading Total Intermittent Generation (MW)',
       'Regions TAS Trading Total Intermittent Generation (MW)',
       'Regions VIC Trading Total Intermittent Generation (MW)',
       'Regions NSW Operational Demand (MW)',
       'Regions SA Operational Demand (MW)',
       'Regions TAS Operational Demand (MW)',
       'Regions VIC Operational Demand (MW)', 'period', 'date'],
      dtype='object')

### Battery Class

In [4]:
###################################
# A Class to store battery functionaly such as revenue, charge and discharge
# periods, charge and discharge spot prices, charge and discharge market dispatch
#
# Consists of:
#      - charge_period : period when it should charge.
#      - discharge_period : period when it should discharge.
#      - charge_price : spot price given charging period.
#      - discharge_price : spot price given discharging period.
#      - charge_market_dispatch : set amount of market dispatch given charging period.
#      - discharge_market_dispatch : set amount of market dispatch given discharging period.
# 
# Functions:
#      - ComputeRevenue: To calculate revenue given discharge and charge period pairs
#
# Created by: Gilbert
###################################
class Battery:
    def __init__(self, charge_period, charge_spot_price,
                 discharge_period, discharge_spot_price):
        
        self.mlf = 0.991 # Set marginal loss factor in the specs.
        
        self.charge_period = charge_period
        self.discharge_period = discharge_period
        
        self.charge_price = charge_spot_price
        self.discharge_price = discharge_spot_price
        
        self.charge_market_dispatch = []
        self.discharge_market_dispatch = []
        
        self.revenue = 0
        
    def ComputeRevenue(self):
        if self.charge_market_dispatch == [] or self.discharge_market_dispatch == []:
            return 0
        
        # Spot Prices
        charge_sp = np.array(self.charge_price)[:, 1]
        discharge_sp = np.array(self.discharge_price)[:, 1]
        
        # Market Dispatches
        charge_md = np.array(self.charge_market_dispatch).T
        discharge_md = np.array(self.discharge_market_dispatch).T
        
        print(charge_sp, charge_md)
        print(discharge_sp, discharge_md)
        
        # Revenues
        charge_revenue = (charge_sp @ charge_md) * (1 / self.mlf)
        discharge_revenue = (discharge_sp @ discharge_md) * (self.mlf)
        
        print(charge_revenue, discharge_revenue)
        return discharge_revenue - charge_revenue

### Helper function for battery optimisation

In [5]:
###################################
# A function to find minimum and maximum point rank given threshold.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#      - buy_threshold : maximum number of buying point, default has been set to optimise Checkpoint 3.
#      - sell_threshold : maximum number of selling point, default has been set to optimise Checkpoint 3.
#
# Return:
#      - List of selected minimum point, list of selected maximum point
#
# Created by: Gilbert
###################################
def GetMinMax(data, region = 'VIC', buy_threshold = 13, sell_threshold = 4):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
        
    price = np.array(spot_price)
    minimum_price = np.argsort(price, kind = 'merge*sort') # (O(NlogN)), mergesort the minimum prices.
    maximum_price = minimum_price[::-1][:48] # (O(N)), maximum is the reverse order of minimum.
    
    selected_min_price = ['' for i in minimum_price] # (O(N)), set an empty array for the whole period.
    selected_max_price = ['' for i in minimum_price] # (O(N)), set an empty array for the whole period.
    
    # Select the lowest price spot over the given
    # buy_threshold as the minimum buying point.
    i = 0
    for b_t in range(buy_threshold):
        selected_min_price[minimum_price[i]] = b_t + 1
        i += 1
        
    # Select the highest price spot over the given
    # sell_threshold as the maximum selling point.
    i = 0
    for s_t in range(sell_threshold):
        selected_max_price[maximum_price[i]] = s_t + 1
        i += 1
        
    return selected_min_price, selected_max_price

###################################
# A function to Find Battery Charge and Discharge pairs in backward order.
# Backward order from 48th period to the 1st.
#
# Parameters:
#      - buy_period : Selected minimum price point as it will be where we buy energy for charging.
#      - sell_period : Selected maximum price point as it will be where we sell energy for discharging.
#
# Return:
#      - List of battery class pairs
#
# Created by: Gilbert
###################################
def FindBatteryPairs(buy_period, sell_period):
    period = 48
        
    battery = []
    sell = OrderedDict() # Initialise battery selling point. (Ordered Dictionary)
    buy = OrderedDict() # Initialise battery buying point. (Orderered Dictionary)
    
    # Iterate over the whole period backwards
    for p in range(period - 1, -1, -1):
        # If maximum selling point is not empty, add (order, period)
        # as key-value pair into the OrderedDict.
        if sell_period[p] != '':
            sell[sell_period[p]] = sell_period.index(sell_period[p]) + 1
        # If battery selling point is not empty and minimum buying 
        # point is not empty.
        if len(sell) != 0 and buy_period[p] != '':
            # If battery buying point period is less than selling 
            # and buying ratio of 4:5 or 1:1.25 (rounded up), then
            # add minimum selling point into Dictionary.
            if len(buy) < math.ceil(len(sell) * 1.25):
                buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
            # else, if battery buying point is full and there is 
            # lower minimum buying point then remove the highest
            # buying point and add the new one into Dictionary.
            else:
                max_key = max(buy, key=buy.get)
                if buy_period[p] < max_key:
                    buy.pop(max_key)
                    buy[buy_period[p]] = buy_period.index(buy_period[p]) + 1
        # If the next period is not empty and battery buying point
        # is not empty then battery charge-discharge pair has been
        # created.
        # Reinitialise a new battery setup.
        if sell_period[p - 1] != '' and len(buy) != 0:
            battery.append([list(sell.items()), list(buy.items())])
            sell = OrderedDict()
            buy = OrderedDict()
    # Add the last battery charge-discharge pair occuring 
    # before 1st period.
    battery.append([list(sell.items()), list(buy.items())]) 

    # Check whether there is too many selling points, then
    # remove selling point until the number of selling points
    # is equal to the number of buying points while removing
    # the lowest selling point.
    for b in battery:
        sell_tmp = np.array(b[0])
        buy_tmp = b[1]
        while len(sell_tmp) > len(buy_tmp): 
            row = 0
            index = np.where(sell_tmp[:,0] == sell_tmp[:,0].max())[0][0]
            sell_tmp = np.delete(sell_tmp, index, axis = row)
        b[0] = sell_tmp.tolist() # Change numpy array to list
        
    return battery

###################################
# A function to get spot prices based on selected regions.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - selected_periods : selected period for charging or discharging.
#      - region : the targeted region, default has been set to 'VIC' for mandatory task.
#
# Return:
#      - List of spot prices given period
#
# Created by: Gilbert
###################################
def GetSpotPrice(data, selected_periods, region = 'VIC'):
    if region == 'VIC':
        spot_price = data['Regions VIC Trading Price ($/MWh)']
    elif region == 'NSW':
        spot_price = data['Regions NSW Trading Price ($/MWh)']
    elif region == 'SA':
        spot_price = data['Regions SA Trading Price ($/MWh)']
    elif region == 'TAS':
        spot_price = data['Regions TAS Trading Price ($/MWh)']
    
    spot_price = np.array(spot_price)
    
    # Find the spot prices from selected region. Periods are
    # index + 1, therefore to use the index we need to subtract
    # it by 1.
    retrieved_prices = []
    for period in selected_periods:
        # append(minimum or maximum ranking, spot_price[index])
        retrieved_prices.append((period[1], spot_price[period[1] - 1]))
    return retrieved_prices

###################################
# A function to set optimal charge and discharge amount of battery pairs.
#
# Parameters:
#      - data : the targeted dataset, minimum dataset length of 48.
#      - battery_pairs : list of all battery class pairs.
#
# Return:
#      - List of all battery class pairs
#
# Created by: Gilbert
###################################
def SetChargeDischarge(data, battery_pairs):
    buy_market_dispatch = (150, 150, 150, 150, 44)
    sell_market_dispatch = (135, 135, 135, 117)
    
    all_batteries = []
    
    battery_pairs = battery_pairs[::-1]
    for b in battery_pairs:
        sell_period = b[0][::-1] # Reverse the order
        buy_period = b[1][::-1] # Reverse the order
        
        sell_price = GetSpotPrice(data, sell_period)
        buy_price = GetSpotPrice(data, buy_period)
        
        battery = Battery(buy_period, buy_price, sell_period, sell_price)
        
        if len(sell_period) == len(sell_market_dispatch) and \
           len(buy_period) == len(buy_market_dispatch):
                
            battery.charge_market_dispatch.extend(buy_market_dispatch)
            battery.discharge_market_dispatch.extend(sell_market_dispatch)
        
        if len(sell_period) < len(buy_period):
            energy_required = 17
            battery.charge_market_dispatch.extend(buy_market_dispatch[:len(sell_period)])
            battery.charge_market_dispatch.append(energy_required)
            
            battery.discharge_market_dispatch.extend(sell_market_dispatch[:len(sell_period)])
        elif len(sell_period) == len(buy_period):
            battery.charge_market_dispatch.extend(buy_market_dispatch[:len(sell_period)])
            battery.discharge_market_dispatch.extend(sell_market_dispatch[:len(sell_period)])
        
        all_batteries.append(battery)
    
    return all_batteries

###################################
# A function to calculate daily revenue.
#
# Parameters:
#      - all_batteries : List of battery class pairs.
#
# Return:
#      - List of battery class pairs
#
# Created by: Gilbert
###################################
def ComputeDailyRevenue(all_batteries):
    revenues = 0
    for battery in all_batteries:
        revenues += battery.ComputeRevenue()
    return revenues

In [6]:
# Test Period checkpoint 3
cp3_start_period = '2020-07-17 00:30:00'
cp3_end_period   = '2020-07-18 00:00:00'

vic_spot_price = market_data[['Time (UTC+10)', 'Regions VIC Trading Price ($/MWh)', 
                              'period']]

cp_3 = vic_spot_price.loc[(vic_spot_price['Time (UTC+10)'] >= cp3_start_period) & \
        (vic_spot_price['Time (UTC+10)'] <= cp3_end_period)]

min_price_cp3, max_price_cp3 = GetMinMax(cp_3)

battery_pairs = FindBatteryPairs(min_price_cp3, max_price_cp3)

all_batteries = SetChargeDischarge(cp_3, battery_pairs)

dailyrev = ComputeDailyRevenue(all_batteries)
print(dailyrev)

[58.04 51.85] [150 150]
[153.81 110.29] [135 135]
16633.198789101916 35332.6185
[39.75 39.94 39.97] [150 150  17]
[163.47 137.1 ] [135 135]
12747.719475277498 40211.75745
46163.45768562058


In [7]:
cp_3

,Time (UTC+10),Regions VIC Trading Price ($/MWh),period
44545,2020-07-17 00:30:00,75.51,1
44546,2020-07-17 01:00:00,73.98,2
44547,2020-07-17 01:30:00,75.57,3
44548,2020-07-17 02:00:00,71.94,4
44549,2020-07-17 02:30:00,74.10,5
44550,2020-07-17 03:00:00,67.36,6
44551,2020-07-17 03:30:00,58.04,7
44552,2020-07-17 04:00:00,51.85,8
44553,2020-07-17 04:30:00,74.53,9
44554,2020-07-17 05:00:00,67.32,10


In [8]:
minima = []
maxima = []

period = 48

start = 0
end = period 
while end <= len(market_data):
    data = market_data.iloc[start:end, :]
    price = data['Regions VIC Trading Price ($/MWh)']
    min_price, max_price = GetMinMax(price)
    tmp_minima, tmp_maxima = BuySellPeriod(min_price, max_price)
    
    minima.extend(tmp_minima)
    maxima.extend(tmp_maxima)
    start += period
    end += period

minima.append(float("nan"))
maxima.append(float("nan"))

market_data['minimum_price'] = pd.Series(minima)
market_data['maximum_price'] = pd.Series(maxima)

KeyError: 'Regions VIC Trading Price ($/MWh)'

In [ ]:
market_data